In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

from pprint import pprint
import json

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\IloveKOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-QISotLL5-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model,
)

In [5]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [6]:
from langchain_core.documents import Document

documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.",
]

doc_objects = []

for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_Textbook", "chapter": f"Chapter {i}"}
    )

    doc_objects.append(doc)

#순차적 ID 리스트 생성
doc_ids = [f"doc_{i}" for i in range(1, len(doc_objects) + 1)]

added_docs_ids = chroma_db.add_documents(documents = doc_objects, ids=doc_ids)

print(f"{len(added_docs_ids)} 개의 문서가 성공적으로 DB에 추가되었습니다.")
print(added_docs_ids)

5 개의 문서가 성공적으로 DB에 추가되었습니다.
['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5']


In [7]:
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2) #관련 문서 k개 추출, 없으면 0도 가능

print(f"질문: {query}")
for doc in results:
    print(f"문서 ID: {doc.metadata['chapter']}, 내용: {doc.page_content}")

질문: 인공지능과 머신러닝의 관계는?
문서 ID: Chapter 2, 내용: 머신러닝은 인공지능의 하위 분야입니다.
문서 ID: Chapter 3, 내용: 딥러닝은 머신러닝의 한 종류입니다.


In [8]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [17]:
# chroma_db.update_documents(document_ids=added_docs_ids, documents=doc_objects, document=doc_objects)
# Document(page_content, metadata={})

update_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다",
    metadata={"source": "AI_Textbook", "chapter": "Chapter 6"}
)

chroma_db.update_document(
    document_id="doc_1",
    document=update_document_1
)

print("문서 업데이트 완료")

문서 업데이트 완료


In [18]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [19]:
chroma_db.delete(ids=["doc_5"])

In [20]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [21]:
chroma_db2 = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model
)

chroma_db2.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}